# MAML
- omniglot dataset example

In [2]:
!pip install higher

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import higher

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import os
os.mkdir('./omniglorot_dataset')

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
!pip install OmniglotNShot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement OmniglotNShot (from versions: none)
ERROR: No matching distribution found for OmniglotNShot


In [11]:
from omniglot_loaders import OmniglotNShot

ModuleNotFoundError: ignored

In [12]:
n_way = 5 # 문자 특별한 문자의 개수 --> 가 나 다 라 마 5way , 가나다라마바사 7way
k_shot = 5 # n_way중 way당 이미지 개수(훈련용) --> 가: (가1, 가2, 가3,가4, 가5), 나: (나1,2,3,4,5)....
k_qry = 15 # n_way중 way당 이미지 개수(검증용) --> 가: (가 1,2,3,4,5,6,7....15)
task_num = 32 # Task : 나라의 언어 ex) 한국어, 중국어, 영어, 일본어...

n_inner_iter = 5 # Task학습에 대한 반복횟수 

In [13]:
data_base = OmniglotNShot(
    './omniglot_data',
    batchsz=task_num,
    n_way=n_way,
    k_shot=k_shot,
    k_query=k_qry,
    imgsz=28,
    )

NameError: ignored

# DB의 train set 해석

In [14]:
x_spt,y_spt,x_qry,y_qry = data_base.next()  #x_spt: 훈련용 데이터 5개 카테고리 + 카테고리당 이미지 5개 y_spt: 훈련용데이터 레이블,
# x_qrt, y_qrt --> validation용도
print(x_spt.shape) # (batch, label, image) 차원 
print(x_qry.shape)
print(y_spt.shape)
print(y_qry.shape)

NameError: ignored

Task개수 : 32개

Task당 

--> train용도의 데이터 (5개 고유문자, 5개씩 추출)

--> validation용도 데이터 (5개 고유문자, 총 75개)

In [ ]:
'''
CNN모델 생성
input 차원
    (채널,가로,세로) --> (1,28,28)

            커널    스트라이드     입력채널        출력채널          출력결과
conv2d      3       1              1               64                (64,26,26)
batchnorm                                          64                (64,26,26)
maxpool2d   2       2                                                (64,13,13)
conv2d      3       1              64              64                (64,11,11)
maxpool2d   2       2                                                (64,5,5)
conv2d      3       1              64              64                (64,3,3)
maxpool2d   2       2                                                (64,1,1)
Flatten                                                              (64)

Fully connected
            입력노드        출력노드
Linear      64              n_way(5)                                 (5)
'''
class Omniglot_model(nn.Module):
    def __init__(self):
        super(Omniglot_model,self).__init__()
        self.sequent = nn.Sequential(
            nn.Conv2d(1,64,3),
            nn.BatchNorm2d(64,momentum=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,64,3),
            nn.BatchNorm2d(64,momentum=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,64,3),
            nn.BatchNorm2d(64,momentum=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        
        self.lin1 = nn.Linear(64,n_way)
            
    def forward(self,x):
        x = self.sequent(x) # CNN
        x = x.view(x.shape[0],-1) # Flatten
        x = self.lin1(x) # F.C
        return x
    

In [15]:
meta_model = Omniglot_model().to(device)
meta_opt = optim.Adam(meta_model.parameters(),lr=1e-3)

NameError: ignored

Train step

In [ ]:
import time 

train_log = []
for epoch in range(3):
    meta_model.train()
    inner_opt = optim.Adam(meta_model.parameters(),lr=1e-1)

    n_train_iter = data_base.x_train.shape[0]//data_base.batchsz
    for batch_idx in range(n_train_iter):
        start_time = time.time()
        x_spt,y_spt,x_qry,y_qry = data_base.next()
        x_spt,x_qry = torch.Tensor(x_spt).to(device), torch.Tensor(x_qry).to(device)
        y_spt,y_qry = torch.LongTensor(y_spt).to(device), torch.LongTensor(y_qry).to(device)

        task_num, setsz, c_, h, w = x_spt.shape
        querysz = x_qry.shape[1]
            
        qry_losses,qry_accs = [],[]
        meta_opt.zero_grad()
        
        # higher library 등장!
        for i in range(task_num):
            with higher.innerloop_ctx(meta_model,inner_opt,copy_initial_weights=False) as (fnet,diffopt):
                    # inner loop
                    for _ in range(n_inner_iter):
                        spt_logits = fnet(x_spt[i])
                        spt_loss = F.cross_entropy(spt_logits,y_spt[i])
                        diffopt.step(spt_loss)
                
            # Knowledge gathering
            qry_logits=fnet(x_qry[i])
            qry_loss = F.cross_entropy(qry_logits,y_qry[i])
            qry_losses.append(qry_loss.detach())
            qry_acc = (qry_logits.argmax(dim=1) == y_qry[i]).sum().item()/querysz
            qry_accs.append(qry_acc)
            
            qry_loss.backward()
        # outer loop    
        meta_opt.step()
        
        qry_losses = sum(qry_losses)/task_num
        qry_accs = 100*sum(qry_accs)/task_num
        i = epoch + float(batch_idx)/n_train_iter
        iter_time = time.time()-start_time
        
        if batch_idx % 4 == 0:
            print('Epoch {:.2f} | Train_loss {:.2f} | Acc: {:.2f} | Time: {:.2f}'\
                 .format(i,qry_losses,qry_accs,iter_time))
        
        train_log.append({'Epoch':i,'Loss':qry_losses,'Accuracy':qry_accs,\
                    'Mode':'train','Time':time.time()})

# Test step

In [ ]:
test_log = []

meta_model.train()
n_test_iter = data_base.x_test.shape[0] // data_base.batchsz
losses,accs = [],[]

for batch_idx in range(n_test_iter):
    x_spt,y_spt,x_qry,y_qry = data_base.next('test')
    x_spt,x_qry = torch.Tensor(x_spt).to(device), torch.Tensor(x_qry).to(device)
    y_spt,y_qry = torch.LongTensor(y_spt).to(device), torch.LongTensor(y_qry).to(device)

    task_num,setsz,c_,h,w = x_spt.shape
    querysz = x_qry.shape[1]
    
    inner_opt = optim.SGD(meta_model.parameters(),lr=1e-1)
    
    # in test step --> no meta_train!
    for i in range(task_num):
        with higher.innerloop_ctx(meta_model,inner_opt,track_higher_grads=False) as (fnet,diffopt):
            # inner train 
            for _ in range(n_inner_iter):
                spt_logits = fnet(x_spt[i])
                spt_loss = F.cross_entropy(spt_logits,y_spt[i])
                diffopt.step(spt_loss)
            
            qry_logits=fnet(x_qry[i]).detach()
            qry_loss = F.cross_entropy(
                    qry_logits,y_qry[i],reduction='none')
            losses.append(qry_loss.detach())
            accs.append((qry_logits.argmax(dim=1)==y_qry[i]).detach())
    
    qry_losses = torch.cat(losses).mean().item()
    qry_accs = 100*torch.cat(accs).float().mean().item()
    
    print('Epoch {:.2f} | Test_loss {:.2f} | Acc: {:.2f}' \
                 .format(i,qry_losses,qry_accs))
    test_log.append({'Epoch':i,'Loss':qry_losses,'Accuracy':qry_accs,\
                    'Mode':'test','Time':time.time()})